In [5]:
week_fp_base = 'nfl-big-data-bowl-2021/week{}.csv'
week_fp = week_fp_base.format(1)
play_fp = 'nfl-big-data-bowl-2021/plays.csv'

In [6]:
import pandas as pd
import numpy as np

week_data = pd.read_csv(week_fp)

In [7]:
week_data.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,gameId,playId,playDirection,route
0,2018-09-07T01:07:14.599Z,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,Matt Ryan,2.0,QB,1,away,2018090600,75,left,NaN
1,2018-09-07T01:07:14.599Z,88.89,36.47,0.01,0.01,0.01,105.63,66.66,None,79848.0,Malcolm Jenkins,27.0,SS,1,home,2018090600,75,left,NaN
2,2018-09-07T01:07:14.599Z,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,Julio Jones,11.0,WR,1,away,2018090600,75,left,HITCH
3,2018-09-07T01:07:14.599Z,86.31,22.01,0.09,0.42,0.01,70.12,168.91,None,2495613.0,Corey Graham,24.0,FS,1,home,2018090600,75,left,NaN
4,2018-09-07T01:07:14.599Z,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,Mohamed Sanu,12.0,WR,1,away,2018090600,75,left,HITCH


In [8]:
week_data.describe()

,x,y,s,a,dis,o,dir,nflId,jerseyNumber,frameId,gameId,playId
count,986022.000000,986022.000000,986022.000000,986022.000000,986022.000000,919109.000000,919109.000000,9.191090e+05,919109.000000,986022.000000,9.860220e+05,986022.000000
mean,58.975056,26.795439,3.187303,2.012526,0.323366,179.545196,181.238304,2.366598e+06,37.763714,35.450674,2.018091e+09,2268.306314
std,24.935961,11.204589,2.843484,1.689410,0.307017,100.706586,102.965112,6.193264e+05,24.977470,23.654286,9.381592e+01,1280.426499
min,-5.980000,-11.740000,0.000000,0.000000,0.000000,0.000000,0.000000,3.060000e+02,1.000000,1.000000,2.018091e+09,58.000000
25%,39.980000,19.000000,0.850000,0.740000,0.090000,89.880000,91.520000,2.533040e+06,20.000000,17.000000,2.018091e+09,1199.000000
50%,59.100000,26.830000,2.790000,1.750000,0.280000,177.430000,181.410000,2.552255e+06,29.000000,33.000000,2.018091e+09,2297.000000
75%,78.190000,34.710000,4.960000,2.940000,0.500000,269.840000,271.350000,2.556480e+06,53.000000,50.000000,2.018091e+09,3315.000000
max,128.380000,62.750000,27.370000,58.330000,7.120000,360.000000,360.000000,2.561485e+06,99.000000,172.000000,2.018091e+09,5511.000000


In [78]:
play_data = pd.read_csv(play_fp)

In [79]:
play_data = play_data[play_data['absoluteYardlineNumber'].notnull()]

In [80]:
play_data.columns

Index(['gameId', 'playId', 'playDescription', 'quarter', 'down', 'yardsToGo',
       'possessionTeam', 'playType', 'yardlineSide', 'yardlineNumber',
       'offenseFormation', 'personnelO', 'defendersInTheBox',
       'numberOfPassRushers', 'personnelD', 'typeDropback',
       'preSnapVisitorScore', 'preSnapHomeScore', 'gameClock',
       'absoluteYardlineNumber', 'penaltyCodes', 'penaltyJerseyNumbers',
       'passResult', 'offensePlayResult', 'playResult', 'epa',
       'isDefensivePI'],
      dtype='object')

In [81]:
week_play = week_data.merge(play_data, on=['gameId', 'playId'])

In [82]:
week_play['time'].dtype

dtype('O')

In [83]:
week_play['time'] = pd.to_datetime(week_play['time'], format='%Y-%m-%dT%H:%M:%S')

In [84]:
week_play['time_diff'] = week_play.groupby(['playId', 'gameId', 'displayName'])['time'].diff()
week_play['time_diff'][week_play['time_diff'].isnull()] = pd.Timedelta(0)
week_play['time_acc_s'] = week_play.groupby(['playId', 'gameId', 'displayName'])['time_diff'].transform(
    lambda x: x.map(lambda x: x.microseconds).cumsum()).div(1e6)

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_70234/3501257125.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_play['time_diff'][week_play['time_diff'].isnull()] = pd.Timedelta(0)


In [85]:
off_def = (week_play
           .groupby(['gameId', 'playId', 'team'])['position']
           .apply(lambda x: 'QB' in x.unique() or 'WR' in x.unique())
           .reset_index()
           .rename({'position': 'off'}, axis=1)
           )
week_off_def = week_play.merge(off_def, on=['gameId', 'playId', 'team'])

In [86]:
week_non_foot = week_off_def[week_off_def['displayName'] != 'Football']

In [87]:
week_non_foot['x_starting'] = week_non_foot.groupby(['gameId', 'playId', 'nflId'])['x'].transform(lambda x: x.iloc[0])

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_70234/863904743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_non_foot['x_starting'] = week_non_foot.groupby(['gameId', 'playId', 'nflId'])['x'].transform(lambda x: x.iloc[0])


In [88]:
week_non_foot['y_starting'] = week_non_foot.groupby(['gameId', 'playId', 'nflId'])['y'].transform(lambda x: x.iloc[0])

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_70234/1564298947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_non_foot['y_starting'] = week_non_foot.groupby(['gameId', 'playId', 'nflId'])['y'].transform(lambda x: x.iloc[0])


In [89]:
week_non_foot['yardline_100'] = week_non_foot['absoluteYardlineNumber'].sub(10)

/var/folders/6v/8jcf0jcn7_vfm2q2r4tz44gh0000gn/T/ipykernel_70234/1130439287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  week_non_foot['yardline_100'] = week_non_foot['absoluteYardlineNumber'].sub(10)


In [90]:
week_off = week_non_foot[week_non_foot['off']].groupby(['gameId', 'playId'])[[
    'x_starting', 'yardline_100', 'absoluteYardlineNumber', 'yardsToGo']].first().reset_index()
week_off['yardline_first'] = np.where(week_off['x_starting'].gt(week_off['absoluteYardlineNumber']),
         week_off['yardline_100'].sub(week_off['yardsToGo']),
         week_off['yardline_100'].add(week_off['yardsToGo']))

In [91]:
week_off.describe()

,gameId,playId,x_starting,yardline_100,absoluteYardlineNumber,yardsToGo,yardline_first
count,9.900000e+02,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000
mean,2.018091e+09,2245.342424,59.209495,49.157576,59.157576,9.045455,48.827273
std,9.576537e+01,1281.510067,24.746130,24.163486,24.163486,4.186134,24.416248
min,2.018091e+09,58.000000,5.790000,1.000000,11.000000,1.000000,0.000000
25%,2.018091e+09,1147.250000,36.435000,28.250000,38.250000,7.000000,34.000000
50%,2.018091e+09,2262.500000,58.255000,48.000000,58.000000,10.000000,49.000000
75%,2.018091e+09,3297.750000,81.217500,70.000000,80.000000,10.000000,65.000000
max,2.018091e+09,5511.000000,110.430000,99.000000,109.000000,30.000000,100.000000


In [92]:
week_off[week_off['absoluteYardlineNumber'].isnull()]

,gameId,playId,x_starting,yardline_100,absoluteYardlineNumber,yardsToGo,yardline_first


In [93]:
week_first = week_non_foot.merge(week_off[['gameId', 'playId', 'yardline_first']], on=['gameId', 'playId'])

In [94]:
week_first

,time,x,y,s,a,dis,o,dir,event,nflId,...,playResult,epa,isDefensivePI,time_diff,time_acc_s,off,x_starting,y_starting,yardline_100,yardline_first
0,2018-09-07 01:07:14.599000+00:00,91.73,26.67,0.00,0.01,0.02,289.57,240.93,None,310.0,...,10,0.261827,False,0 days 00:00:00,0.0,True,91.73,26.67,80.0,65.0
1,2018-09-07 01:07:14.599000+00:00,91.35,44.16,0.02,0.03,0.01,290.45,16.86,None,2495454.0,...,10,0.261827,False,0 days 00:00:00,0.0,True,91.35,44.16,80.0,65.0
2,2018-09-07 01:07:14.599000+00:00,90.78,36.15,0.00,0.00,0.00,257.61,193.97,None,2533040.0,...,10,0.261827,False,0 days 00:00:00,0.0,True,90.78,36.15,80.0,65.0
3,2018-09-07 01:07:14.599000+00:00,98.25,26.63,0.01,0.01,0.00,265.88,194.51,None,2543583.0,...,10,0.261827,False,0 days 00:00:00,0.0,True,98.25,26.63,80.0,65.0
4,2018-09-07 01:07:14.599000+00:00,91.29,21.83,0.01,0.02,0.01,250.34,151.83,None,2555415.0,...,10,0.261827,False,0 days 00:00:00,0.0,True,91.29,21.83,80.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879415,2018-09-11 05:29:39.900000+00:00,25.02,36.59,4.65,1.28,0.47,49.06,12.98,None,2543492.0,...,8,-2.418280,False,0 days 00:00:00.101000,6.4,False,23.73,18.80,31.0,29.0
879416,2018-09-11 05:29:39.900000+00:00,23.66,20.13,2.63,2.49,0.27,24.32,12.77,None,2552488.0,...,8,-2.418280,False,0 days 00:00:00.101000,6.4,False,29.43,7.64,31.0,29.0
879417,2018-09-11 05:29:39.900000+00:00,33.68,43.14,1.96,3.79,0.22,281.35,309.70,None,2556277.0,...,8,-2.418280,False,0 days 00:00:00.101000,6.4,False,35.90,24.39,31.0,29.0
879418,2018-09-11 05:29:39.900000+00:00,36.02,40.21,0.81,1.52,0.09,308.07,234.06,None,2556593.0,...,8,-2.418280,False,0 days 00:00:00.101000,6.4,False,36.93,36.39,31.0,29.0
